In [1]:
import pandas as pd

In [5]:
large = False

In [6]:
results = pd.read_csv('./results_large.csv' if large else './results.csv', names=['name', 'repo', 'acc', '#w', 'samples', 'time', 'model', 'model_address', 'split'])
results.tail(1)

,name,repo,acc,#w,samples,time,model,model_address,split
181,normal,/data/all/data/foam-framework/foam,0.33,13,33,29-04-2022_22-17-41,t5-small,./storage//tmp/bt//data/all/data/foam-framewor...,repo-based-included


In [7]:
results = results[results['split'] != 0.5]
results = results[results['model'] == f't5-{"large" if large else "small"}']

In [9]:
def extract_type_from_address(address):
    for keyword in ['bt', '_length_label', '_conf', 'included', 'ft', 'basic', 'finetuned', 'adapted']:
        if keyword in address:
            if keyword == 'ft' and '0.3_4' in address:
                return 'ft_0.3_4'
            return keyword
    return 'excluded'

In [10]:
results['type'] = results['model_address'].apply(extract_type_from_address)

In [11]:
results['type'].value_counts()

finetuned        40
adapted          30
ft               20
ft_0.3_4         20
_conf            20
_length_label    20
excluded         11
included         11
bt               10
Name: type, dtype: int64

In [12]:
# results[results['type'] == 'ft']

In [13]:
results.tail(3)

,name,repo,acc,#w,samples,time,model,model_address,split,type
179,normal,/data/all/data/LivelyKernel/LivelyKernel,0.58,12,36,29-04-2022_22-16-05,t5-small,./storage//tmp/bt//data/all/data/LivelyKernel/...,repo-based-included,bt
180,normal,/data/all/data/svgdotjs/svg.js,0.10,5,31,29-04-2022_22-16-58,t5-small,./storage//tmp/bt//data/all/data/svgdotjs/svg.js,repo-based-included,bt
181,normal,/data/all/data/foam-framework/foam,0.33,13,33,29-04-2022_22-17-41,t5-small,./storage//tmp/bt//data/all/data/foam-framewor...,repo-based-included,bt


In [14]:
f'{10:.2f}, {1:.2f}'

'10.00, 1.00'

In [15]:
def print_stats(df):
    print(f"{len(df)}, {100 * (df['acc'] * df['samples']).sum() / df['samples'].sum():.2f}, {100 * df['acc'].mean():.2f}, {100 * df['acc'].median():.2f}")

In [16]:
print_stats(results[(results['type'] == 'included') & (results['repo'] != 'all')])

10, 51.61, 50.90, 49.00


In [17]:
print_stats(results[(results['type'] == 'excluded') & (results['repo'] != 'all')])

10, 49.47, 49.30, 45.50


In [18]:
print_stats(results[(results['type'] == 'finetuned') & (results['repo'] != 'all')])

20, 58.67, 60.40, 54.50


In [19]:
print_stats(results[(results['type'] == 'finetuned') & (results['repo'] == 'all')])

20, 40.85, 40.85, 42.50


In [20]:
print_stats(results[(results['type'] == 'adapted') & (results['repo'] != 'all')])

20, 59.28, 60.40, 57.00


In [21]:
print_stats(results[(results['type'] == 'ft') & (results['repo'] != 'all')])

10, 49.31, 48.40, 47.00


In [22]:
print_stats(results[(results['type'] == 'curr') & (results['repo'] != 'all')])

0, nan, nan, nan


/tmp/ipykernel_115071/2078206976.py:2: RuntimeWarning: invalid value encountered in double_scalars
  print(f"{len(df)}, {100 * (df['acc'] * df['samples']).sum() / df['samples'].sum():.2f}, {100 * df['acc'].mean():.2f}, {100 * df['acc'].median():.2f}")


In [23]:
print_stats(results[(results['type'] == 'curr') & (results['repo'] == 'all')])

0, nan, nan, nan


/tmp/ipykernel_115071/2078206976.py:2: RuntimeWarning: invalid value encountered in double_scalars
  print(f"{len(df)}, {100 * (df['acc'] * df['samples']).sum() / df['samples'].sum():.2f}, {100 * df['acc'].mean():.2f}, {100 * df['acc'].median():.2f}")


In [24]:
print_stats(results[(results['type'] == 'adapted') & (results['repo'] == 'all')])

10, 41.00, 41.00, 43.00


In [25]:
print_stats(results[(results['type'] == 'adapted') & (results['repo'] != 'all') & results['split'].isna()])

10, 59.61, 60.80, 57.00


In [26]:
print_stats(results[(results['type'] == 'adapted') & (results['repo'] != 'all') & ~results['split'].isna()])

10, 58.96, 60.00, 55.00


In [27]:
print_stats(results[(results['type'] == 'ft_0.3_4') & (results['repo'] != 'all')])

10, 49.31, 48.40, 47.00


In [28]:
print_stats(results[(results['type'] == 'ft_0.3_4') & (results['repo'] == 'all')])

10, 43.20, 43.20, 43.00


In [29]:
print_stats(results[(results['type'] == 'ft') & (results['repo'] == 'all')])

10, 43.20, 43.20, 43.00


In [30]:
print_stats(results[(results['type'] == 'ft') & (results['repo'] != 'all')])

10, 49.31, 48.40, 47.00


In [31]:
print_stats(results[(results['type'] == '_conf') & (results['repo'] == 'all')])

10, 43.80, 43.80, 44.00


In [32]:
print_stats(results[(results['type'] == '_conf') & (results['repo'] != 'all')])

10, 53.34, 54.90, 50.00


In [33]:
print_stats(results[(results['type'] == '_length_label') & (results['repo'] != 'all')])

10, 55.97, 58.00, 51.00


In [36]:
print_stats(results[(results['type'] == '_length_label') & (results['repo'] == 'all')])

10, 43.00, 43.00, 44.00


In [38]:
print_stats(results[(results['type'] == 'bt') & (results['repo'] != 'all')])

10, 50.89, 50.40, 44.00


In [35]:
results[(results['type'] == 'adapted') & (results['repo'] != 'all') & results['split'].isna()]

,name,repo,acc,#w,samples,time,model,model_address,split,type
22,withAdapter,/data/all/data/qooxdoo/qooxdoo,0.57,18,115,25-01-2022_18-37-47,t5-small,/scratch/arminz//tmp/adapted,NaN,adapted
23,withAdapter,/data/all/data/elastic/kibana,0.47,17,51,25-01-2022_18-41-01,t5-small,/scratch/arminz//tmp/adapted,NaN,adapted
24,withAdapter,/data/all/data/emberjs/ember.js,0.57,11,47,25-01-2022_19-40-30,t5-small,/scratch/arminz//tmp/adapted,NaN,adapted
25,withAdapter,/data/all/data/zloirock/core-js,0.49,8,41,25-01-2022_20-09-49,t5-small,/scratch/arminz//tmp/adapted,NaN,adapted
26,withAdapter,/data/all/data/Encapsule-Annex/onm,1.00,1,39,25-01-2022_18-50-58,t5-small,/scratch/arminz//tmp/adapted,NaN,adapted
27,withAdapter,/data/all/data/sequelize/sequelize,0.35,12,40,25-01-2022_18-53-57,t5-small,/scratch/arminz//tmp/adapted,NaN,adapted
28,withAdapter,/data/all/data/dcos/dcos-ui,0.86,9,37,25-01-2022_18-57-25,t5-small,/scratch/arminz//tmp/adapted,NaN,adapted
29,withAdapter,/data/all/data/LivelyKernel/LivelyKernel,0.67,12,36,25-01-2022_19-00-44,t5-small,/scratch/arminz//tmp/adapted,NaN,adapted
30,withAdapter,/data/all/data/svgdotjs/svg.js,0.74,5,31,25-01-2022_22-10-19,t5-small,/scratch/arminz//tmp/adapted,NaN,adapted
31,withAdapter,/data/all/data/foam-framework/foam,0.36,13,33,25-01-2022_19-07-00,t5-small,/scratch/arminz//tmp/adapted,NaN,adapted


In [467]:
results[(results['type'] == 'adapted') & (results['repo'] != 'all') & results['split'].isna()]

,name,repo,acc,#w,samples,time,model,model_address,split,type
22,withAdapter,/data/all/data/qooxdoo/qooxdoo,0.63,18,115,31-01-2022_17-30-12,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
23,withAdapter,/data/all/data/elastic/kibana,0.59,17,51,31-01-2022_17-39-08,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
24,withAdapter,/data/all/data/emberjs/ember.js,0.57,11,47,31-01-2022_17-47-51,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
25,withAdapter,/data/all/data/Encapsule-Annex/onm,1.00,1,39,31-01-2022_18-05-38,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
26,withAdapter,/data/all/data/dcos/dcos-ui,0.92,9,37,31-01-2022_18-22-39,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
27,withAdapter,/data/all/data/svgdotjs/svg.js,0.84,5,31,31-01-2022_18-38-53,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
28,withAdapter,/data/all/data/foam-framework/foam,0.39,13,33,31-01-2022_18-47-10,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
29,withAdapter,/data/all/data/sequelize/sequelize,0.50,12,40,31-01-2022_20-23-31,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
30,normal,/data/all/data/foam-framework/foam,0.42,13,33,31-01-2022_131-01-2022_20-32-47,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted
31,withAdapter,/data/all/data/zloirock/core-js,0.59,8,41,01-02-2022_15-31-56,t5-large,/scratch/arminz//tmp/adapted,NaN,adapted


In [346]:
from matplotlib import pyplot as plt

In [347]:
plt.scatter(inc_2['samples'], inc_2['acc'], label='included')
plt.scatter(exc_2['samples'], exc_2['acc'], label='excluded')
plt.scatter(good_2['samples'], good_2['acc'], label='tuned')
plt.scatter(adapter['samples'], adapter['acc'], label='tuned with adapter')
plt.legend()

NameError: name 'inc_2' is not defined

In [ ]:
diff_inc = inc_2.set_index('repo').join(inc_2.set_index('repo')['acc'] - exc_2.set_index('repo')['acc'], rsuffix='ـdiff')

In [ ]:
diff_good = good_2.set_index('repo').join(good_2.set_index('repo')['acc'] - exc_2.set_index('repo')['acc'], rsuffix='ـdiff')

In [ ]:
diff_adapter = adapter.set_index('repo').join(adapter.set_index('repo')['acc'] - exc_2.set_index('repo')['acc'], rsuffix='ـdiff')

In [ ]:
plt.scatter(diff_inc['samples'], diff_inc['acc'], label='included')
plt.scatter(diff_adapter['samples'], diff_adapter['acc'], label='adapter')
plt.legend()

In [ ]:
plt.scatter(diff_good['samples'], diff_good['acc'], label='good')


In [ ]:
plt.scatter(diff_adapter['samples'], diff_adapter['acc'], label='adapter')
